# Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.auto import tqdm
import os

## For Google Colab Users

This cell is for mounting your Google Drive to the Colab Notebook. If you are not using Google Colab, you can skip this cell

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Check for GPU in mac
# device = "mps" if torch.backends.mps.is_available() else "cpu"

device

'cuda'

# Data 

## Transforming Data

In [3]:
data_transforms = {

    'Training' : transforms.Compose([
        transforms.RandomResizedCrop((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'Testing': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()

    ])
}


## Loading Data

In [4]:
# directory: where training and testing data are
base_path = os.getcwd()
data_dir = os.path.join(base_path, 'Dataset/')
dir = './data2/'
### START CODE HERE

# datasets.ImageFolder: (https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html)
# torch.utils.data.DataLoader: (https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

# image_datasets are dictionary of (type of dataset, dataloader)
# type of dataset are training and testing
image_datasets = {x: datasets.ImageFolder(os.path.join(dir, x), data_transforms[x]) for x in ['Training','Testing']}

# DataLoader helps us for better performance and experience in data loading
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['Training','Testing']}
### END CODE HERE

dataset_sizes = {x: len(image_datasets[x]) for x in ['Training','Testing']}
class_names = image_datasets['Training'].classes

dataset_sizes, class_names

({'Training': 2770, 'Testing': 394},
 ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor'])

## Samples of data

In [ ]:
samples, labels = next(iter(dataloaders['Testing']))
plt.figure(figsize=(17, 10))
plt.axis('off')
for i in range(10):
    plt.subplot(4, 8, i+1)
    plt.imshow(samples[i].permute(1, 2, 0))
    plt.title(class_names[labels[i]])
    plt.axis('off')

# Model

## Loading Model

In [5]:
# Loading are pretrained model in this task our model is resnet50 (https://www.youtube.com/watch?v=mGMpHyiN5lk)
### START CODE HERE

# Loading pretrained model
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
### END CODE HERE
model

C:\Users\Arian\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Arian\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Arian/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
  4%|▍         | 4.30M/97.8M [00:13<04:53, 334kB/s]


KeyboardInterrupt: 

## Preparing Model

In [ ]:
### START CODE HERE

# You have to change the (fc) layer of the model to compatible with your data
model.fc = nn.Linear(model.fc.in_features, 4)

new_kernel_size = 3
model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=new_kernel_size, stride=2, padding=3, bias=False)

for module in model.modules():
    if isinstance(module, torch.nn.Conv2d):
        module.kernel_size = (new_kernel_size, new_kernel_size)
### END CODE HERE
model = model.to(device)
model

# Training

## Loss function

In [ ]:
criterion = nn.CrossEntropyLoss()

## Optimizer

In [ ]:
# you have to change it for better performance
optimizer = optim.Adam(model.parameters(), lr=0.01)

## Others

In [ ]:
# you can have other thongs like learning rate scheduler and ...

## Train

In [ ]:
### START CODE HERE

losses = []
EPOCH = 10

# for training part you have to set model to train mode
model.train()

# loop on epochs
for e in tqdm(range(EPOCH)):

  # loop on batches
  for inputs, labels in dataloaders['Training']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # set the grad to zero
    optimizer.zero_grad()
    
    # forward part
    # hint: using of pytorch max method (https://pytorch.org/docs/stable/generated/torch.max.html)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)

    #  compute loss
    loss = criterion(outputs, labels)
    
    # backward part
    loss.backward()

    # update parameters
    optimizer.step()

  # you have to append loss for each epoch
  losses.append(loss.item())
### END CODE HERE

## Plot loss function

In [ ]:
# you have to calculate losses arrayin Train part
plt.plot(list(range(len(losses))), losses)
plt.show()

## Evaluate model

In [ ]:
### START CODE HERE

def calc_accuracy(data, model):
  corrects = 0

  # for testing part you have to set model to eval mode
  model.eval()
  for inputs, labels in tqdm(dataloaders[data]):
      inputs = inputs.to(device)
      labels = labels.to(device)
      
      with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        corrects += torch.sum(preds == labels.data)
  return corrects.double() / dataset_sizes[data]

### END CODE HERE

In [ ]:
# accuracy of training data
calc_accuracy("Training", model)

In [ ]:
# accuracy of testing data
calc_accuracy('Testing', model)

# Saving Model

In [ ]:
PATH = os.path.join(base_path, 'model.ci')
torch.save(model, PATH)

# Loading and eval Model

In [ ]:
### START CODE HERE

model_for_eval = torch.load(PATH)
model_for_eval.to(device)

### END CODE HERE

In [ ]:
model_for_eval

In [ ]:
# accuracy of training data by loadded model
calc_accuracy('Training', model_for_eval)

In [ ]:
# accuracy of testing data by loadded model
calc_accuracy('Testing', model_for_eval)